## import

In [48]:
import requests
import pandas as pd
from math import radians, cos, sin, asin, sqrt

## 串接 google map api

In [2]:
f = open("API_KEY.txt", "r")
API_KEY = f.read()

In [12]:
def get_location_information(address,API_KEY,language="zh-TW"):
    req_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    my_params = {
        "address" : address,
        "key" : API_KEY ,
        "language" : language
    }
    r = requests.get(req_url,params=my_params)
    res = r.json()
    data = {}
    if(res['status']=="OK"):
        data['address'] = res['results'][0]['formatted_address']
        data['geocode'] = res['results'][0]['geometry']['location']
        for spec in res['results'][0]['address_components']:
            if spec['types'][0]=="administrative_area_level_3":
                data['place'] = spec['long_name']
    elif(res['status']=='ZERO_RESULTS'):
        data['message'] = "zero results"
    elif(res['status']=='INVALID_REQUEST') :
        data['message'] = res['error_message']
    return {"status":res['status'],"data":data}

In [74]:
get_location_information("捷運忠孝新生站",API_KEY)

{'data': {'address': '106台灣台北市大安區新生南路一段67號10652忠孝新生站',
  'geocode': {'lat': 25.0417882, 'lng': 121.5328524},
  'place': '大安區'},
 'status': 'OK'}

## 撈取捷運站資料

In [15]:
r = requests.get('https://gist.githubusercontent.com/lackneets/f41836e7e4e938a89f73/raw/e427e8c7b58fcce1338c96bc38b890ae2db6a98a/mrt-taipei.json')

In [16]:
mrt_data = r.json()

In [108]:
mrt_data

[{'address': '115台灣台北市南港區捷運南港軟體園區站',
  'id': '030',
  'latitude': 25.059904,
  'line': '文山內湖線',
  'longitude': 121.615952,
  'name': '南港軟體園區'},
 {'address': '115台灣台北市南港區捷運南港展覽館站',
  'id': '031',
  'latitude': 25.055368,
  'line': '文山內湖線',
  'longitude': 121.6176,
  'name': '南港展覽館'},
 {'address': '114台灣台北市內湖區捷運東湖站',
  'id': '029',
  'latitude': 25.067247,
  'line': '文山內湖線',
  'longitude': 121.611504,
  'name': '東湖'},
 {'address': '114台灣台北市內湖區捷運葫洲站',
  'id': '028',
  'latitude': 25.072701,
  'line': '文山內湖線',
  'longitude': 121.607158,
  'name': '葫洲'},
 {'address': '114台灣台北市內湖區捷運大湖公園站',
  'id': '027',
  'latitude': 25.083844,
  'line': '文山內湖線',
  'longitude': 121.60214,
  'name': '大湖公園'},
 {'address': '114台灣台北市內湖區捷運內湖站',
  'id': '026',
  'latitude': 25.08366,
  'line': '文山內湖線',
  'longitude': 121.594407,
  'name': '內湖'},
 {'address': '114台灣台北市內湖區捷運文德站',
  'id': '025',
  'latitude': 25.078531,
  'line': '文山內湖線',
  'longitude': 121.584761,
  'name': '文德'},
 {'address': '114台灣台北市內湖區捷運港墘站',
 

## 去掉重複的捷運站

In [113]:
dup = []
for i,data in enumerate(mrt_data) :
    for j,datay in enumerate(mrt_data) :
        if(i!=j and data['name']==datay['name'] and (i not in dup)):
            dup.append(j)
new_mrt_data = [x for idx,x in enumerate(mrt_data) if (idx not in dup)]

## 將捷運站加入區域中

In [116]:
data_arr = []
print("Having ",len(new_mrt_data)," to process.")
for idx,mrt in enumerate(new_mrt_data):
    name = "捷運"+mrt['name']+"站"
    print("Now processing data ",idx," name : ",name)
    try : 
        data = get_location_information("捷運"+mrt['name']+"站",API_KEY)
        place = data['data']['place']
    except :
        data = get_location_information(mrt['name']+"站",API_KEY)
        place = data['data']['place']
    data_arr.append({"name":name,"lat":mrt['latitude'],"lng":mrt['longitude'],"place":place })

Having  103  to process.
Now processing data  0  name :  捷運南港軟體園區站
Now processing data  1  name :  捷運南港展覽館站
Now processing data  2  name :  捷運東湖站
Now processing data  3  name :  捷運葫洲站
Now processing data  4  name :  捷運大湖公園站
Now processing data  5  name :  捷運內湖站
Now processing data  6  name :  捷運文德站
Now processing data  7  name :  捷運港墘站
Now processing data  8  name :  捷運西湖站
Now processing data  9  name :  捷運劍南路站
Now processing data  10  name :  捷運大直站
Now processing data  11  name :  捷運松山機場站
Now processing data  12  name :  捷運中山國中站
Now processing data  13  name :  捷運南京東路站
Now processing data  14  name :  捷運忠孝復興站
Now processing data  15  name :  捷運大安站
Now processing data  16  name :  捷運科技大樓站
Now processing data  17  name :  捷運六張犁站
Now processing data  18  name :  捷運麟光站
Now processing data  19  name :  捷運辛亥站
Now processing data  20  name :  捷運萬芳醫院站
Now processing data  21  name :  捷運萬芳社區站
Now processing data  22  name :  捷運木柵站
Now processing data  23  name :  捷運動物園站
Now processing data  24

In [117]:
mrt_df = pd.DataFrame(data_arr)

In [118]:
mrt_df

,lat,lng,name,place
0,25.059904,121.615952,捷運南港軟體園區站,南港區
1,25.055368,121.617600,捷運南港展覽館站,南港區
2,25.067247,121.611504,捷運東湖站,內湖區
3,25.072701,121.607158,捷運葫洲站,內湖區
4,25.083844,121.602140,捷運大湖公園站,內湖區
5,25.083660,121.594407,捷運內湖站,內湖區
6,25.078531,121.584761,捷運文德站,內湖區
7,25.080027,121.575080,捷運港墘站,內湖區
8,25.082133,121.567213,捷運西湖站,內湖區
9,25.084853,121.555592,捷運劍南路站,中山區


In [119]:
mrt_df.loc[mrt_df.place == "大安區"]

,lat,lng,name,place
14,25.041629,121.543767,捷運忠孝復興站,大安區
15,25.033254,121.543565,捷運大安站,大安區
16,25.026124,121.543437,捷運科技大樓站,大安區
17,25.023739,121.553004,捷運六張犁站,大安區
18,25.018535,121.558791,捷運麟光站,大安區
46,25.033847,121.528739,捷運東門站,大安區
47,25.033534,121.534869,捷運大安森林公園站,大安區
48,25.033116,121.552798,捷運信義安和站,大安區
52,25.026356,121.522872,捷運古亭站,大安區
66,25.042355,121.532904,捷運忠孝新生站,大安區


## 加總各區域捷運站數量

In [120]:
mrt_gb = mrt_df.groupby(by='place').size()

In [121]:
mrt_count_df = pd.DataFrame(mrt_gb).reset_index()

In [122]:
mrt_count_df = mrt_count_df.rename(columns={0:"count"})

In [123]:
mrt_count_df

,place,count
0,三重區,6
1,中和區,3
2,中山區,7
3,中正區,7
4,信義區,5
5,內湖區,7
6,北投區,9
7,南港區,5
8,土城區,3
9,士林區,3


## 儲存資料至CSV

In [127]:
mrt_df.to_csv("捷運站總表區域.csv",encoding='utf_8_sig')

In [125]:
mrt_count_df.to_csv("每區捷運站數量.csv",encoding='utf_8_sig')

# calculate distance with geocode

In [98]:
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r * 1000

In [79]:
haversine(121.4362613,24.9669358,121.567213,25.082133)

18389.336632420665

In [96]:
def distance_google(origin,destination,API_KEY,language='zh-TW'):
    map_url = "https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial"
    map_params = {
        "origins" : origin,
        "destinations" : destination,
        "key" : API_KEY , 
        "language" : language
    }
    r = requests.get(map_url,map_params)
    res = r.json()
    return r.json()['rows'][0]['elements'][0]

In [97]:
distance_google("霧峰區大同路52巷","永寧捷運站",API_KEY)

{'distance': {'text': '108 哩', 'value': 173069},
 'duration': {'text': '1 小時 47 分', 'value': 6440},
 'status': 'OK'}